In [1]:
import requests

# Specify the URL of the page you want to download
url = "https://pokemondb.net/pokedex/all"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the content to an HTML file
    with open("Pokemon.html", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("Page saved as 'page.html'")
else:
    print(f"Failed to download page. Status code: {response.status_code}")

Page saved as 'page.html'


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URL der Webseite
url = "https://pokemondb.net/pokedex/all"

# HTTP-Anfrage senden
response = requests.get(url)
response.raise_for_status()  # Fehler auslösen, falls die Anfrage fehlschlägt

# HTML-Inhalt der Seite parsen
soup = BeautifulSoup(response.text, 'html.parser')

# Tabelle finden
table = soup.find('table', {'id': 'pokedex'})

# Tabellenüberschriften extrahieren
headers = [header.text for header in table.find('thead').find_all('th')]

# Tabelleninhalte extrahieren
rows = []
for row in table.find('tbody').find_all('tr'):
    cells = row.find_all(['th', 'td'])
    rows.append([cell.text.strip() for cell in cells])

# DataFrame erstellen
df = pd.DataFrame(rows, columns=headers)

# Ausgabe des DataFrames
print(df.head())

# Optional: DataFrame in eine CSV-Datei speichern
df.to_csv("pokemon_data.csv", index=False)


      #                    Name          Type Total  HP Attack Defense  \
0  0001               Bulbasaur  Grass Poison   318  45     49      49   
1  0002                 Ivysaur  Grass Poison   405  60     62      63   
2  0003                Venusaur  Grass Poison   525  80     82      83   
3  0003  Venusaur Mega Venusaur  Grass Poison   625  80    100     123   
4  0004              Charmander          Fire   309  39     52      43   

  Sp. Atk Sp. Def Speed  
0      65      65    45  
1      80      80    60  
2     100     100    80  
3     122     120    80  
4      60      50    65  


In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URL of the webpage
url = "https://pokemondb.net/pokedex/all"

# HTTP request
response = requests.get(url)
response.raise_for_status()  # Raise error if the request fails

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table
table = soup.find('table', {'id': 'pokedex'})

# Extract table headers
headers = [header.text for header in table.find('thead').find_all('th')]

# Extract table rows
rows = []
for row in table.find('tbody').find_all('tr'):
    cells = row.find_all(['th', 'td'])
    rows.append([cell.text.strip() for cell in cells])

# Create a DataFrame
df = pd.DataFrame(rows, columns=headers)

# Convert numerical columns to proper data types for analysis
df['#'] = pd.to_numeric(df['#'], errors='coerce')
df['Total'] = pd.to_numeric(df['Total'], errors='coerce')
df['HP'] = pd.to_numeric(df['HP'], errors='coerce')
df['Attack'] = pd.to_numeric(df['Attack'], errors='coerce')
df['Defense'] = pd.to_numeric(df['Defense'], errors='coerce')
df['Sp. Atk'] = pd.to_numeric(df['Sp. Atk'], errors='coerce')
df['Sp. Def'] = pd.to_numeric(df['Sp. Def'], errors='coerce')
df['Speed'] = pd.to_numeric(df['Speed'], errors='coerce')

# Split types for better analysis
df['Type 1'] = df['Type'].str.split().str[0]
df['Type 2'] = df['Type'].str.split().str[1]

# Helper function to print analysis results
def print_analysis_results(title, results):
    print(f"\n{title}\n{'-' * len(title)}")
    print(results)


In [4]:
# 2a. Stärksten Pokemon von jedem Typ
strongest_overall = df.nlargest(5, 'Total')[['Name', 'Total', 'Type 1', 'Type 2']]
print_analysis_results("Strongest Pokémon (Overall)", strongest_overall)

strongest_by_type = df.loc[df.groupby('Type 1')['Total'].idxmax()][['Type 1', 'Name', 'Total']]
print_analysis_results("Strongest Pokémon by Type", strongest_by_type)


Strongest Pokémon (Overall)
---------------------------
                        Name  Total   Type 1    Type 2
1058     Eternatus Eternamax   1125   Poison    Dragon
201     Mewtwo Mega Mewtwo X    780  Psychic  Fighting
202     Mewtwo Mega Mewtwo Y    780  Psychic       NaN
475   Rayquaza Mega Rayquaza    780   Dragon    Flying
471     Kyogre Primal Kyogre    770    Water       NaN

Strongest Pokémon by Type
-------------------------
        Type 1                      Name  Total
168        Bug        Pinsir Mega Pinsir    600
862       Dark                   Yveltal    680
475     Dragon    Rayquaza Mega Rayquaza    780
1192  Electric                  Miraidon    670
1054     Fairy      Zacian Crowned Sword    700
1056  Fighting  Zamazenta Crowned Shield    700
314       Fire                     Ho-oh    680
767     Flying  Tornadus Incarnate Forme    580
597      Ghost    Giratina Altered Forme    680
319      Grass    Sceptile Mega Sceptile    630
473     Ground    Groudon Primal

In [5]:
# 2b. Stärksten Angreifer
best_attackers = df.nlargest(5, 'Attack')[['Name', 'Attack', 'Type 1', 'Type 2']]
print_analysis_results("Best Attackers", best_attackers)


Best Attackers
--------------
                         Name  Attack   Type 1    Type 2
201      Mewtwo Mega Mewtwo X     190  Psychic  Fighting
274  Heracross Mega Heracross     185      Bug  Fighting
956                   Kartana     181    Grass     Steel
473    Groudon Primal Groudon     180   Ground      Fire
475    Rayquaza Mega Rayquaza     180   Dragon    Flying


In [6]:
# 2c. Der Durschnitt von jedem Pokemon Typ
average_stats_by_type = df.groupby('Type 1')[['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].mean()
print_analysis_results("Average Stats by Type", average_stats_by_type)


Average Stats by Type
---------------------
               Total         HP      Attack     Defense    Sp. Atk    Sp. Def  \
Type 1                                                                          
Bug       381.164835  56.802198   70.912088   71.208791  55.549451  64.835165   
Dark      449.714286  72.892857   85.482143   70.982143  72.267857  70.553571   
Dragon    527.551020  84.571429  103.816327   80.816327  90.122449  83.571429   
Electric  449.405405  63.837838   73.148649   65.837838  88.675676  70.405405   
Fairy     449.225806  72.129032   71.064516   73.645161  78.129032  87.193548   
Fighting  458.080000  75.740000  104.960000   76.440000  55.160000  69.660000   
Fire      456.986842  70.763158   84.473684   69.315789  86.828947  71.644737   
Flying    450.500000  70.900000   81.900000   67.400000  72.600000  70.900000   
Ghost     438.276596  63.595745   72.000000   78.042553  82.361702  78.978723   
Grass     426.460177  67.318584   77.743363   72.318584  75.3539

In [7]:
# Optional: Save cleaned DataFrame to CSV
df.to_csv("pokemon_data_cleaned.csv", index=False)